In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [2]:
df_train = pd.read_parquet('AQI_Historical_New')
df_test = pd.read_parquet('AQI_Historical_Dec_2022.parquet')

In [3]:
df_train = df_train[df_train['TRT_ID'] == 20]
df_test = df_test[df_test['TRT_ID'] == 20]

In [4]:
df_train = df_train.drop('coord.lon', axis=1)
df_train = df_train.drop('coord.lat', axis=1)
df_train = df_train.drop('TRT_ID', axis=1)
df_test = df_test.drop('coord.lon', axis=1)
df_test = df_test.drop('coord.lat', axis=1)
df_test = df_test.drop('TRT_ID', axis=1)

In [5]:
df_train["dt"] = pd.to_datetime(df_train["dt"])
df_test["dt"] = pd.to_datetime(df_test["dt"])

In [6]:
df = df_train

In [7]:
from prophet import Prophet

In [8]:
df_train = df_train.rename(columns={'dt':'ds', 'components.pm2_5':'y'})
df_train['y_orig'] = df_train['y']
df_train['y'] = np.log(df_train['y'])

In [9]:
model_new = Prophet(interval_width=0.95)
model_new.add_regressor('components.co')
model_new.add_regressor('components.no')
model_new.add_regressor('components.no2')
model_new.add_regressor('components.o3')
model_new.add_regressor('components.so2')
model_new.add_regressor('components.pm10')
model_new.add_regressor('components.nh3')

In [10]:
model_new.fit(df_train)

12:56:32 - cmdstanpy - INFO - Chain [1] start processing
12:56:39 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
future_data = model_new.make_future_dataframe(periods=30, freq='D')

In [12]:
df = df.append(df_test)
df = df.rename(columns={'dt':'ds', 'components.pm2_5':'y'})
future_data = df[['ds','components.co', 'components.no', 'components.no2', 'components.o3', 'components.so2', 'components.pm10', 'components.nh3']]

C:\Users\fashaikh\AppData\Local\Temp\ipykernel_18404\92394862.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_test)


In [13]:
forecast_data = model_new.predict(future_data)
forecast_data[['ds','yhat','yhat_lower','yhat_upper']].tail(5)

,ds,yhat,yhat_lower,yhat_upper
17299,2022-12-31 19:00:00,1.758171,0.971993,2.558978
17300,2022-12-31 20:00:00,1.831230,1.025805,2.685188
17301,2022-12-31 21:00:00,1.857412,1.042465,2.700333
17302,2022-12-31 22:00:00,1.967330,1.161009,2.793849
17303,2022-12-31 23:00:00,2.117423,1.321366,3.019953


In [14]:
forecast_data_orig = forecast_data
forecast_data_orig['yhat'] = np.exp(forecast_data_orig['yhat'])
forecast_data_orig['yhat_lower'] = np.exp(forecast_data_orig['yhat_lower'])
forecast_data_orig['yhat_upper'] = np.exp(forecast_data_orig['yhat_upper'])

In [15]:
df_train['y_log'] = df_train['y']
df_train['y'] = df_train['y_orig']

In [16]:
final_df = pd.DataFrame(forecast_data_orig)
import plotly.graph_objs as go
import plotly.offline as py

actual_chart = go.Scatter(y=df_train['y_orig'], name='Actual')
predicted_chart = go.Scatter(y=final_df['yhat'], name='Predicted')
predicted_chart_upper = go.Scatter(y=final_df['yhat_upper'], name='Predicted Upper')
predicted_chart_lower = go.Scatter(y=final_df['yhat_lower'], name='Predicted Lower')
#real_actual = go.Scatter(y=df_test['components.pm2_5'] ,name='Real Actual')

py.plot([actual_chart,predicted_chart,predicted_chart_upper,predicted_chart_lower])

'temp-plot.html'